In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import wget
import os
from datetime import datetime
import requests

In [8]:
# baixa preços cripto
def crip_quotation(symbol, ini_date, comparison_symbol='BRL', all_data=True, limit=1, aggregate=1, exchange=''):
    from datetime import datetime
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    if all_data:
        url += '&allData=true'
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.fromtimestamp(d) for d in df.time]
    
    df['timestamp'] = df['timestamp'].dt.normalize()
    df = df.set_index('timestamp').drop(['time','high','low','open','volumefrom','volumeto','conversionType','conversionSymbol'],axis=1)
    df = df[ini_date:]
    
    return df

In [9]:
# recebe caminho de um arquivo como parâmetro, devolve dataframe com papeis e cotacoes
def baixa_cotacoes(arquivo,cripto=False):
    df = pd.read_csv(arquivo, delimiter=";")
    df["ini"] = pd.to_datetime(df["ini"], format="%d/%m/%Y")
    df.set_index("acao", inplace=True)
    cotacoes = pd.DataFrame()
    for acao in df.index:
        ini = df.loc[acao,"ini"]
        print(acao, ini)
        if cripto == True:
            cotacoes[acao] = crip_quotation(acao, ini_date=ini)["Adj Close"]
        elif cripto == False:
            cotacoes[acao] = yf.download(acao, start=ini)["Close"] # +".SA"
    return(cotacoes)

In [10]:
# baixa renda fixa
def precos_rf():
    df = pd.DataFrame()
    rf = pd.read_excel('lista rf.xlsx')
    for i in range(0,len(rf)):
        link = rf['download'][i] # link para download do arquivo de rf
        arquivo = rf['arquivo'][i]
        aba = rf['ativo'][i] # o nome da aba é o nome do ativo
        ini_date = rf['data ini'][i]
        # baixa dados
        wget.download(link)
        td = pd.read_excel(arquivo, aba,skiprows=[0]).set_index('Dia') # skiprows é uma lista de linhas para pular. No caso pula a linha 0
        os.remove(arquivo) # deleta depois de baixar
        td = td[ini_date:] # filtra só as linhas a partir da data desejada
        # transforma pro formato datetime
        td = td.set_index(pd.DataFrame([datetime.strptime(date, '%d/%m/%Y').date() for date in td.index[:]])[0]) 
        td = td['PU Compra Manhã']  # filtra a coluna de preço correta
        df[aba] = td # cria uma coluna com o nome do ativo e o preço histórico
    return(df)

In [11]:
df = baixa_cotacoes("lista acoes.csv")
df.to_excel("./resultados/cotacoes acao.xlsx")
df

VALE3.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
KLBN11.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
MOVI3.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
PNVL3.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
RRRP3.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
TTEN3.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
LAVV3.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
ALSO3.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
YDUQ3.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
TUPY3.SA 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
MGLU3.SA 2022-05-02

,VALE3.SA,KLBN11.SA,MOVI3.SA,PNVL3.SA,RRRP3.SA,TTEN3.SA,LAVV3.SA,ALSO3.SA,YDUQ3.SA,TUPY3.SA,MGLU3.SA,BBAS3.SA
Date,,,,,,,,,,,,
2022-05-02,82.919998,20.959999,16.930000,11.70,43.869999,8.95,4.76,20.440001,15.78,20.540001,4.80,32.980000
2022-05-03,82.500000,21.219999,16.129999,11.45,43.560001,8.88,4.81,20.049999,15.80,20.590000,4.60,33.660000
2022-05-04,81.809998,22.240000,16.969999,11.63,45.599998,9.32,4.98,20.110001,16.50,21.160000,4.95,34.639999
2022-05-05,80.330002,22.500000,16.090000,11.18,44.380001,8.91,4.87,19.170000,15.56,21.070000,4.42,33.740002
2022-05-06,79.760002,22.370001,15.550000,10.96,44.000000,8.34,4.86,18.629999,15.12,20.530001,4.30,33.849998
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-27,70.519997,19.459999,12.770000,10.47,30.670000,9.14,4.62,16.870001,13.84,22.270000,2.70,35.549999
2022-07-28,70.690002,19.230000,12.900000,11.06,31.830000,9.12,4.88,17.070000,13.53,22.389999,2.72,36.209999
2022-07-29,69.750000,19.930000,12.700000,10.98,34.020000,9.18,4.86,16.870001,13.25,22.770000,2.58,35.970001


In [11]:
df = baixa_cotacoes("lista acoes US.csv")
df.to_excel("./resultados/cotacoes acao US.xlsx")
df

SPY 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
AAXJ 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
XLU 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed


,SPY,AAXJ,XLU
Date,,,
2022-05-02,412.697266,70.681290,69.970062
2022-05-03,414.589111,71.258598,70.237923
2022-05-04,427.214569,71.945404,71.805374
2022-05-05,412.030151,69.247948,71.071251
2022-05-06,409.570770,68.292397,71.636726
...,...,...,...
2022-07-27,401.040009,68.519997,70.779999
2022-07-28,406.070007,68.550003,73.320000
2022-07-29,411.989990,68.000000,73.949997


In [14]:
df = precos_rf()
df.to_excel("./resultados/cotacoes rf.xlsx")
df

100% [..............................................................................] 94208 / 94208

,LTN 010125,LFT 010327,NTN-B Princ 150535
0,,,
2022-05-02,735.84,11503.77,1930.89
2022-05-03,734.44,11509.38,1924.80
2022-05-04,733.90,11515.28,1906.99
2022-05-05,735.98,11521.80,1933.96
2022-05-06,731.99,11528.71,1914.91
...,...,...,...
2022-07-27,739.41,11865.10,1831.88
2022-07-28,742.31,11871.00,1842.95
2022-07-29,745.85,11877.00,1846.64


In [15]:
df = baixa_cotacoes("lista commodities.csv")
df.to_excel("./resultados/cotacoes commodities.xlsx")
df

SLV 2022-05-01 00:00:00
[*********************100%***********************]  1 of 1 completed
GSG 2022-05-01 00:00:00
[*********************100%***********************]  1 of 1 completed
COPX 2022-05-01 00:00:00
[*********************100%***********************]  1 of 1 completed
LIT 2022-05-01 00:00:00
[*********************100%***********************]  1 of 1 completed


,SLV,GSG,COPX,LIT
Date,,,,
2022-05-02,20.870001,24.059999,38.640076,65.605957
2022-05-03,20.830000,23.559999,39.060608,66.625114
2022-05-04,21.280001,24.430000,39.901672,68.843285
2022-05-05,20.780001,24.330000,37.838131,66.984818
2022-05-06,20.660000,24.370001,37.143764,66.884903
...,...,...,...,...
2022-07-27,17.600000,22.860001,28.900000,74.089996
2022-07-28,18.430000,22.790001,29.209999,74.330002
2022-07-29,18.700001,22.930000,30.490000,74.389999


In [16]:
df = baixa_cotacoes("lista hedge.csv")
df.to_excel("./resultados/cotacoes hedge.xlsx")
df

GLD 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed


,GLD
Date,
2022-05-02,173.630005
2022-05-03,174.089996
2022-05-04,175.800003
2022-05-05,175.130005
2022-05-06,175.419998
...,...
2022-07-27,161.669998
2022-07-28,163.639999
2022-07-29,164.100006


In [17]:
cotacoes = baixa_cotacoes("lista cripto.csv", cripto=True)
cotacoes.to_excel("./resultados/cotacoes cripto.xlsx")
cotacoes

BTC 2022-05-02 00:00:00
ETH 2022-05-02 00:00:00
DOT 2022-05-02 00:00:00
ALGO 2022-05-02 00:00:00
MATIC 2022-05-02 00:00:00


,BTC,ETH,DOT,ALGO,MATIC
timestamp,,,,,
2022-05-02,188398.91,13890.65,73.45,3.344,5.367
2022-05-03,195713.13,14498.43,80.54,3.526,5.778
2022-05-04,185037.49,13909.27,73.77,3.214,5.383
2022-05-05,184045.43,13761.08,73.20,3.554,5.327
2022-05-06,181693.05,13497.98,70.43,3.837,5.221
...,...,...,...,...,...
2022-07-28,123822.14,8968.44,42.66,1.746,4.882
2022-07-29,123244.64,8844.04,42.77,1.758,4.846
2022-07-30,121882.78,8782.87,45.05,1.755,4.844
